In [3]:
# cleaning from numbers 
# 2.6 mln images

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter


#df = pd.read_csv('dm1/dm_clean1.csv')

In [88]:
# delete link to image
df = df.drop(['photoLink'], axis=1)

In [89]:
# delete numbers from tags
df['photoTags'] = df['photoTags'].str.replace("\d+", '')
#for the spaces
df['photoTags'] = df['photoTags'].str.replace("sec", '')
df['photoTags'] = df['photoTags'].str.replace("nabkv", '')

In [90]:
# delete values less then 3
df['photoTags'] = df['photoTags'].str.split().map(lambda sl: " ".join(s for s in sl if len(s) > 3))
# delete geolat geolon
df['photoTags'] = df['photoTags'].str.replace("geolat", '')
df['photoTags'] = df['photoTags'].str.replace("geolon", '')
df['photoTags'] = df['photoTags'].str.replace("geotagged", '')
df['photoTags'] = df['photoTags'].str.strip()

In [91]:
# check if there are some Nan values after cleaning
df['photoTags'].replace('', np.nan, inplace=True)
df = df.dropna(subset=['photoTags'])

In [86]:
print(df)

            photoID  latitude  longitude        userID  \
0        1151334893     45.33       5.95  75174621@N00   
1        1151332465     45.33       5.96  75174621@N00   
2        1438670393     40.75      14.49  13897419@N02   
3        1438658045     40.75      14.49  13897419@N02   
4        1439150203     42.64      18.12  13897419@N02   
5        1439384711     50.94       6.95  13897419@N02   
6         199700867     32.74    -117.15  71102643@N00   
7        1294940523     41.40       2.16  86354760@N00   
8        1444197150     50.85       4.35  13897419@N02   
9        1445262436     59.44      24.79  13897419@N02   
10       1444357105     59.44      24.74  13897419@N02   
11       1444291815     52.36       4.89  13897419@N02   
12       1802417334     40.74     -73.99  13897419@N02   
13       1440252476     48.14      11.58  13897419@N02   
14       1447581462     40.75     -73.99  13897419@N02   
15       1294901725     41.40       2.17  86354760@N00   
16        1380

In [92]:
# save data
df.to_csv("dm_clean3.csv", encoding="utf-8",index=None)

In [4]:
# we have cleaned training data 
train = pd.read_csv('dm_clean3.csv')

In [6]:
# we have test data
test = pd.read_csv('test5_all.csv')
# clean the same way
# delete link to image
test = test.drop(['photoLink'], axis=1)
test['photoTags'] = test['photoTags'].str.replace("\d+", '')
test['photoTags'] = test['photoTags'].str.replace("sec", '')
test['photoTags'] = test['photoTags'].str.replace("nabkv", '')
# delete values less then 3
test = test[test['photoTags'].str.len() > 3]
# delete geolat geolon
test['photoTags'] = test['photoTags'].str.replace("geolat", '')
test['photoTags'] = test['photoTags'].str.replace("geolon", '')
test['photoTags'] = test['photoTags'].str.replace("geotagged", '')
# check if there are some Nan values after cleaning
test['photoTags'].replace('', np.nan, inplace=True)
test = test.dropna(subset=['photoTags'])

In [95]:
print(len(test))

215441


In [7]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
# some images without tags have 0 prediction 
# create corpus
# so we have features 
# load data 
crp = pd.read_csv('corpus.csv')
#take tags from the column 'photoTags'
tags = crp.loc[:,'words']

tfidf = TfidfVectorizer()
tf_matrix = tfidf.fit_transform(tags)

In [8]:
vocab = tfidf.vocabulary_

In [18]:
# we have sparse matrix for 3 mln pictures 
# create KNN
from sklearn.neighbors import NearestNeighbors
train_data = pd.read_csv('dm_clean3.csv')
train_data_ = train_data.loc[:,'photoTags']
_vect = TfidfVectorizer(vocabulary = vocab)
train_c_matrix = _vect.fit_transform(train_data_)

nbrs = NearestNeighbors(n_neighbors = 1, algorithm='ball_tree')
nbrs.fit(train_c_matrix)

C:\Apps\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:211: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=1, p=2, radius=1.0)

In [19]:
test_data = pd.read_csv('test5_all.csv')
# delete all data with empty tags from test data
test_data = test_data.dropna(subset=['photoTags'])
# save it 
test_data.to_csv("test5_all_clean.csv", encoding="utf-8",index=None)
# load data again
test_data = pd.read_csv('test5_all_clean.csv')
test_data_ = test_data.loc[:,'photoTags']
new_vect = TfidfVectorizer(vocabulary = vocab)
test_c_matrix = new_vect.fit_transform(test_data_)

In [20]:
data_lt = pd.DataFrame([], columns=['lt'])
data_ln = pd.DataFrame([], columns=['ln'])

for i in range(0,36):
    distances, indices = nbrs.kneighbors(test_c_matrix[i])

    ind = indices[0][0]
    print('ind',ind)
    print(distances)
    
    lt_test = test_data.loc[i,'latitude']
    ln_test = test_data.loc[i,'longitude']
    
    lt = train_data.loc[ind,'latitude']
    lng = train_data.loc[ind,'longitude']
    
    print('test:', lt_test, ln_test)
    print('train:', lt, lng)    
    
#    df_lt = pd.DataFrame([lt], columns=['lt'])
#    df_ln = pd.DataFrame([lng], columns=['ln'])
    
#    data_lt = data_lt.append(df_lt)
#    data_ln = data_ln.append(df_ln)
    print('test:', test_data.loc[i,'photoTags'])
    print('train:', train_data.loc[ind,'photoTags'])
    print('-----------------------------')


ind 107452
[[ 0.]]
test: 38.659016 -57.767097
train: 40.72 -73.99
test: fortune pei
train: fortune
-----------------------------
ind 139954
[[ 0.89079541]]
test: 47.52434 -52.622547
train: 47.52 -52.62
test: chimney rust stack cape spear
train: cape spear east
-----------------------------
ind 1816560
[[ 0.45168905]]
test: 47.57131 -52.680965
train: 47.57 -52.68
test: sunset car newfoundland lights stjohns signalhill cabottower
train: sign newfoundland stjohns signalhill cabottower fallhazard
-----------------------------
ind 274011
[[ 0.70794809]]
test: 64.09141 -72.1582
train: 51.28 0.52
test: wifi broadband
train: broadband carphonewarehouse
-----------------------------
ind 84
[[ 1.]]
test: -31.637306 129.43114
train: -29.9 -71.25
test: geotagged coast australia cliffs sa southaustralia bundacliffs greataustralianbight geo:lat=31637306 geo:lon=129431133
train: universidaddelmar
-----------------------------
ind 2439836
[[ 0.86516245]]
test: -31.71634 128.89006
train: -31.65 128.97


In [106]:
print(test_data)

           photoID   latitude   longitude  accuracy           userID  \
0       1472189240  38.659016  -57.767097      16.0     31911001@N00   
1       1441071302  47.524340  -52.622547      16.0     12739532@N08   
2       1438580776  47.571310  -52.680965      16.0     12739532@N08   
3        156031551  64.091410  -72.158200      16.0     94366076@N00   
4        168780851 -31.637306  129.431140      16.0     73115147@N00   
5        159794254 -31.716340  128.890060      16.0     73115147@N00   
6       1458844834  -3.112405  -60.023804      16.0     13052927@N00   
7        197009486  12.338857   45.435110      16.0     37473564@N00   
8        197006833  12.335330   45.438100      16.0     37473564@N00   
9       1817002237  52.244408  -10.102202      16.0     72498751@N00   
10      1936992667  52.245193  -10.097165      16.0     72498751@N00   
11      1766178928  52.243057  -10.131787      16.0     72498751@N00   
12      1479578708  52.244408  -10.102202      16.0     72498751